# Project 2


# Movie Genre Classification

Classify a movie genre based on its plot.

<img src="https://github.com/andresmarin94/Repositorio_Personal/blob/master/moviegenre.png?raw=1"
     style="float: left; margin-right: 10px;" />




https://www.kaggle.com/c/miia4200-20191-p2-moviegenreclassification/overview


### Data

Input:
- movie plot

Output:
Probability of the movie belong to each genre


### Evaluation

- 20% API
- 30% Create a solution using with a Machine Learning algorithm - Presentation (5 slides)
- 50% Performance in the Kaggle competition (Normalized acording to class performance in the private leaderboard)


### Acknowledgements

We thank Professor Fabio Gonzalez, Ph.D. and his student John Arevalo for providing this dataset.

See https://arxiv.org/abs/1702.01992

## Sample Submission

In [38]:
import pandas as pd
import os
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.multiclass import OneVsRestClassifier
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier,AdaBoostClassifier,GradientBoostingClassifier,BaggingClassifier
from sklearn.metrics import r2_score, roc_auc_score
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from nltk.stem import WordNetLemmatizer
import nltk
nltk.download('wordnet')
import xgboost
from sklearn.linear_model import LogisticRegression, LogisticRegressionCV

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\carlo\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [39]:
import re

In [40]:
dataTraining = pd.read_csv('https://github.com/albahnsen/AdvancedMethodsDataAnalysisClass/raw/master/datasets/dataTraining.zip', encoding='UTF-8', index_col=0)
dataTesting = pd.read_csv('https://github.com/albahnsen/AdvancedMethodsDataAnalysisClass/raw/master/datasets/dataTesting.zip', encoding='UTF-8', index_col=0)

### Create count vectorizer


In [41]:
dataTraining.head(5)

,year,title,plot,genres,rating
3107,2003,Most,most is the story of a single father who takes...,"['Short', 'Drama']",8.0
900,2008,How to Be a Serial Killer,a serial killer decides to teach the secrets o...,"['Comedy', 'Crime', 'Horror']",5.6
6724,1941,A Woman's Face,"in sweden , a female blackmailer with a disfi...","['Drama', 'Film-Noir', 'Thriller']",7.2
4704,1954,Executive Suite,"in a friday afternoon in new york , the presi...",['Drama'],7.4
2582,1990,Narrow Margin,"in los angeles , the editor of a publishing h...","['Action', 'Crime', 'Thriller']",6.6


In [42]:
dataTesting.head(5)

,year,title,plot
1,1999,Message in a Bottle,"who meets by fate , shall be sealed by fate ...."
4,1978,Midnight Express,"the true story of billy hayes , an american c..."
5,1996,Primal Fear,martin vail left the chicago da ' s office to ...
6,1950,Crisis,husband and wife americans dr . eugene and mr...
7,1959,The Tingler,the coroner and scientist dr . warren chapin ...


In [43]:
dataTraining['title']=dataTraining['title'].str.lower()
dataTraining['title']=dataTraining['title'].str.replace(',', '') 
dataTraining['title']=dataTraining['title'].str.replace('.', '') 
dataTraining['title']=dataTraining['title'].str.replace('-', '') 
dataTraining['title']=dataTraining['title'].str.replace('_', '') 
dataTraining['plot']=dataTraining['plot'].str.lower()
dataTraining['plot']=dataTraining['plot'].str.replace(',', '') 
dataTraining['plot']=dataTraining['plot'].str.replace('.', '') 
dataTraining['plot']=dataTraining['plot'].str.replace('-', '') 
dataTraining['plot']=dataTraining['plot'].str.replace('_', '') 

In [44]:
dataTesting['plot']=dataTesting['plot'].str.lower()
dataTesting['plot']=dataTesting['plot'].str.replace(',', '') 
dataTesting['plot']=dataTesting['plot'].str.replace('.', '') 
dataTesting['plot']=dataTesting['plot'].str.replace('-', '') 
dataTesting['plot']=dataTesting['plot'].str.replace('_', '')
dataTesting['title']=dataTesting['title'].str.lower()
dataTesting['title']=dataTesting['title'].str.replace(',', '') 
dataTesting['title']=dataTesting['title'].str.replace('.', '') 
dataTesting['title']=dataTesting['title'].str.replace('-', '') 
dataTesting['title']=dataTesting['title'].str.replace('_', '') 

In [45]:
wordnet_lemmatizer = WordNetLemmatizer()

In [46]:
dataTraining['uno']= dataTraining['title']+dataTraining['plot']
dataTesting['uno'] = dataTesting['title']+dataTesting['plot']

data = dataTraining['uno'].append(dataTesting['uno'])
years = dataTraining['year'].append(dataTesting['year'])

In [47]:
wordnet_lemmatizer = WordNetLemmatizer()
def split_into_lemmas(text):
    text = text.lower()
    words = text.split()
    return [wordnet_lemmatizer.lemmatize(word, 's') for word in words]

vect = TfidfVectorizer(analyzer=split_into_lemmas, min_df=0.5,max_features=None,sublinear_tf=True,
                       strip_accents='unicode',ngram_range=(1,1),stop_words='english')

#X_dtm = vect.fit_transform(dataTraining['uno'],dataTraining['year'])
#X_dtm.shape
X_dtm = vect.fit_transform(data, years)

In [48]:
X_dtm_test = X_dtm[dataTraining.shape[0]:,:]
X_dtm = X_dtm[:dataTraining.shape[0],:]

### Create y

In [49]:
dataTraining['genres'] = dataTraining['genres'].map(lambda x: eval(x))

le = MultiLabelBinarizer()
y_genres = le.fit_transform(dataTraining['genres'])

In [50]:
X_train, X_test, y_train_genres, y_test_genres = train_test_split(X_dtm, y_genres, test_size=0.3, random_state=666)

### Train multi-class multi-label model

In [51]:

#clf = OneVsRestClassifier(RandomForestClassifier(n_jobs=-1,max_depth=10,n_estimators=1000, random_state=60,criterion='entropy'))

In [52]:
LogReg_Model = OneVsRestClassifier(LogisticRegression(multi_class='multinomial',solver='newton-cg',C=1.5,
#                                                     warm_start=True,penalty='elasticnet',random_state=456, cv=10))
                                                      warm_start=True, penalty='l2',random_state=999), n_jobs=-1)

LogReg_Model.fit(X_dtm, y_genres)
y_pred_genres = LogReg_Model.predict_proba(X_test)
roc_auc_score(y_test_genres, y_pred_genres, average='macro')

0.9978753586152531

### Predict the testing dataset

In [53]:


cols = ['p_Action', 'p_Adventure', 'p_Animation', 'p_Biography', 'p_Comedy', 'p_Crime', 'p_Documentary', 'p_Drama', 'p_Family',
        'p_Fantasy', 'p_Film-Noir', 'p_History', 'p_Horror', 'p_Music', 'p_Musical', 'p_Mystery', 'p_News', 'p_Romance',
        'p_Sci-Fi', 'p_Short', 'p_Sport', 'p_Thriller', 'p_War', 'p_Western']

y_pred_test_genres = LogReg_Model.predict_proba(X_dtm_test)


In [54]:
res = pd.DataFrame(y_pred_test_genres, index=dataTesting.index, columns=cols)

In [55]:
res.head()

,p_Action,p_Adventure,p_Animation,p_Biography,p_Comedy,p_Crime,p_Documentary,p_Drama,p_Family,p_Fantasy,...,p_Musical,p_Mystery,p_News,p_Romance,p_Sci-Fi,p_Short,p_Sport,p_Thriller,p_War,p_Western
1,0.127745,0.075168,0.025105,0.014199,0.355707,0.091853,0.007531,0.394088,0.043238,0.127143,...,0.041594,0.062683,0.000802,0.735588,0.022803,0.007067,0.012123,0.115414,0.014115,0.020544
4,0.130193,0.029207,0.025150,0.122078,0.212179,0.331181,0.045155,0.863114,0.035065,0.020349,...,0.022171,0.039258,0.001085,0.058271,0.014037,0.011146,0.021364,0.352303,0.033377,0.014370
5,0.051110,0.013522,0.004275,0.053790,0.056144,0.741449,0.011720,0.798634,0.007635,0.017998,...,0.013826,0.469070,0.000995,0.128030,0.025960,0.003984,0.019306,0.599722,0.019309,0.009918
6,0.091516,0.088869,0.010174,0.040652,0.147693,0.051997,0.009011,0.798654,0.027789,0.040580,...,0.017791,0.085374,0.000826,0.145891,0.112813,0.005591,0.022607,0.378228,0.068839,0.019514
7,0.027789,0.035458,0.019342,0.025383,0.153218,0.073329,0.014743,0.382080,0.033705,0.106052,...,0.016976,0.081116,0.000843,0.134363,0.394684,0.008970,0.008558,0.352654,0.010251,0.013471


In [56]:
res.to_csv('mindf1.csv', index_label='ID')